#  EYE TRACKING AND BLINK RATIO CALCULATION using MediaPipe model and OpenCv

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import math
import time
import matplotlib as plt

In [3]:


# variables...
frame_counter = 0
cef_counter = 0
total_blinks = 0

BLACK = (0,0,0)
WHITE = (255,255,255)
BLUE = (255,0,0)
RED = (0,0,255)
CYAN = (255,255,0)
YELLOW =(0,255,255)
MAGENTA = (255,0,255)
GRAY = (128,128,128)
GREEN = (0,255,0)
PURPLE = (128,0,128)
ORANGE = (0,165,255)
PINK = (147,20,255)

# face bounder indices 
# FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]

# lips indices for Landmarks
LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 

# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

mp_face_mesh = mp.solutions.face_mesh
cam = cv2.VideoCapture("D:/Emotion_recognition/archive/emotion.mp4")

# function for landmarks detection... 
def LandmarksDetection(image,results,draw=False):
    img_h, img_w,_ = image.shape
    # for point in results.multi_face_landmarks:
    #     mesh_points = [(int(point.x * img_w),int(point.y * img_h))]
    mesh_points = [(int(point.x * img_w), int(point.y * img_h)) for point in results.multi_face_landmarks[0].landmark]
    if draw:
        [cv2.circle(image, p, 2, (0,255,0), -1) for p in mesh_points]
        
    return mesh_points
# function for calculating eucleadian distance...
def eucldian_distance(X,Y):
    x1,y1 = X
    x2,y2 = Y
    dis = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return dis
# function for calculation of blinking ratio...
def blinkratio(image,landmarks,right_indices,left_indices):
    # draw horizontal line on right eye
    right_rh = landmarks[right_indices[0]]
    left_rh = landmarks[right_indices[8]]
    # draw vertical line on right eye
    top_rv = landmarks[right_indices[12]]
    bottom_rv = landmarks[right_indices[4]]
    # draw horizontal line on left eye
    right_lh = landmarks[left_indices[0]]
    left_lh = landmarks[left_indices[8]]
    # draw vertical line on left eye
    top_lv = landmarks[left_indices[12]]
    bottom_lv = landmarks[left_indices[4]]
    
    right_h_dis = eucldian_distance(right_rh,left_rh)
    right_v_dis = eucldian_distance(top_rv,bottom_rv)
    
    left_h_dis = eucldian_distance(right_lh,left_lh)
    left_v_dis = eucldian_distance(top_lv,bottom_lv)
    
    right_eye_ratio = right_h_dis/right_v_dis
    left_eye_ratio = left_h_dis/left_v_dis
    
    blink_ratio = (right_eye_ratio+left_eye_ratio)/2
    
    return blink_ratio

def eyesExtractor(image,r_eye_points,l_eye_points):
    # converting color image to gray scale image
    gray_img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    img_size = gray_img.shape
    mask = np.zeros(img_size,dtype=np.uint8)
    cv2.fillPoly(mask,[np.array(r_eye_points,dtype=np.int32)],255)
    cv2.fillPoly(mask,[np.array(l_eye_points,dtype=np.int32)],255)
    
    eyes = cv2.bitwise_and(gray_img,gray_img,mask=mask)
    eyes[mask==0]=155
    r_max_x = (max(r_eye_points, key=lambda item: item[0]))[0]
    r_min_x = (min(r_eye_points, key=lambda item: item[0]))[0]
    r_max_y = (max(r_eye_points, key=lambda item : item[1]))[1]
    r_min_y = (min(r_eye_points, key=lambda item: item[1]))[1]
    
    
    l_max_x = (max(l_eye_points, key=lambda item: item[0]))[0]
    l_min_x = (min(l_eye_points, key=lambda item: item[0]))[0]
    l_max_y = (max(l_eye_points, key=lambda item : item[1]))[1]
    l_min_y = (min(l_eye_points, key=lambda item: item[1]))[1]

    # croping the eyes from mask 
    cropped_right = eyes[r_min_y: r_max_y, r_min_x: r_max_x]
    cropped_left = eyes[l_min_y: l_max_y, l_min_x: l_max_x]

    # returning the cropped eyes 
    return cropped_right, cropped_left

def EyePositionEstimator(eye):
    h,w = eye.shape
    guass_blur = cv2.GaussianBlur(eye,(9,9),0)
    median_blur = cv2.medianBlur(guass_blur,3)
    
    ret, threshed_eye = cv2.threshold(median_blur,130,255,cv2.THRESH_BINARY)
    fxd_part = int(w/3)
    
    right_eye_part = threshed_eye[0:h,0:fxd_part]
    center_eye_part = threshed_eye[0:h,fxd_part:fxd_part+fxd_part]
    left_eye_part = threshed_eye[0:h,fxd_part+fxd_part:w]
    
    eye_position, color = pixelCounter(right_eye_part,center_eye_part,left_eye_part)
    return eye_position, color

def pixelCounter(first_part,second_part,third_part):
    right_eye_part = np.sum(first_part==0)
    center_eye_part = np.sum(second_part==0)
    left_eye_part = np.sum(third_part==0)
    
    eye_parts = [right_eye_part,center_eye_part,left_eye_part]
    
    max_ind = eye_parts.index(max(eye_parts))
    pos_eye = ""
    if max_ind ==0:
        pos_eye = "RIGHT"
        color = [BLACK,GREEN]
    elif max_ind == 1:
        pos_eye = "CENTER"
        color = [YELLOW,PINK]
    elif max_ind == 2:
        pos_eye = "LEFT"
        color = [GRAY,YELLOW]
    else:
        pos_eye = "CLOSED"
        color = [GRAY,YELLOW]
    return pos_eye, color

def textWithBackground(img, text, font, fontScale, textPos, textThickness=1,textColor=(0,255,0), bgColor=(0,0,0), pad_x=3, pad_y=3, bgOpacity=0.5):
    (t_w, t_h), _= cv2.getTextSize(text, font, fontScale, textThickness) # getting the text size
    x, y = textPos
    overlay = img.copy() # coping the image
    cv2.rectangle(overlay, (x-pad_x, y+ pad_y), (x+t_w+pad_x, y-t_h-pad_y), bgColor,-1) # draw rectangle 
    new_img = cv2.addWeighted(overlay, bgOpacity, img, 1 - bgOpacity, 0) # overlaying the rectangle on the image.
    cv2.putText(new_img,text, textPos,font, fontScale, textColor,textThickness ) # draw in text
    img = new_img

    return img

def colorBackgroundText(img, text, font, fontScale, textPos, textThickness=1,textColor=(0,255,0), bgColor=(0,0,0), pad_x=3, pad_y=3):
    (t_w, t_h), _= cv2.getTextSize(text, font, fontScale, textThickness) # getting the text size
    x, y = textPos
    cv2.rectangle(img, (x-pad_x, y+ pad_y), (x+t_w+pad_x, y-t_h-pad_y), bgColor,-1) # draw rectangle 
    cv2.putText(img,text, textPos,font, fontScale, textColor,textThickness ) # draw in text

    return img



CLOSED_EYES_FRAME =3
FONTS =cv2.FONT_HERSHEY_COMPLEX
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5,min_tracking_confidence=0.5,max_num_faces=4) as face_mesh:
    start_time = time.time()
    while True:
        frame_counter += 1
        ret,frame = cam.read()
        if not ret:
            break
        frame = cv2.resize(frame,None,fx = 1.5, fy=1.5,interpolation=cv2.INTER_CUBIC)
        frame_h, frame_w = frame.shape[:2]
        rgb_img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_img)
        if results.multi_face_landmarks:
            mesh_points = LandmarksDetection(rgb_img,results,False)
            ratio = blinkratio(rgb_img,mesh_points,RIGHT_EYE,LEFT_EYE)
            colorBackgroundText(rgb_img,  f'Ratio : {round(ratio,2)}', FONTS, 0.7, (30,100),2, PINK, YELLOW)
            
            if ratio > 5.5:
                cef_counter += 1
                colorBackgroundText(rgb_img,  f'Blink', FONTS, 1.7, (int(frame_h/2), 100), 2, YELLOW, pad_x=6, pad_y=6, )
                
            else:
                if cef_counter > CLOSED_EYES_FRAME:
                    total_blinks += 1
                    cef_counter = 0
            colorBackgroundText(rgb_img,  f'Total Blinks: {total_blinks}', FONTS, 0.7, (30,150),2)
            cv2.polylines(rgb_img,  [np.array([mesh_points[p] for p in LEFT_EYE ], dtype=np.int32)], True, GREEN, 1, cv2.LINE_AA)
            cv2.polylines(rgb_img,  [np.array([mesh_points[p] for p in RIGHT_EYE ], dtype=np.int32)], True, GREEN, 1, cv2.LINE_AA)
            cv2.polylines(rgb_img,  [np.array([mesh_points[p]for p in [33,133]], dtype=np.int32)], True, BLUE, 1, cv2.LINE_AA)
            cv2.polylines(rgb_img,  [np.array([mesh_points[p]for p in [362,263]], dtype=np.int32)], True, BLUE, 1, cv2.LINE_AA)
            cv2.polylines(rgb_img,  [np.array([mesh_points[p]for p in [145,159]], dtype=np.int32)], True, BLUE, 1, cv2.LINE_AA)
            cv2.polylines(rgb_img,  [np.array([mesh_points[p]for p in [386,374]], dtype=np.int32)], True, BLUE, 1, cv2.LINE_AA)
            # Blink Detector Counter Completed
            right_coords = [mesh_points[p] for p in RIGHT_EYE]
            left_coords = [mesh_points[p] for p in LEFT_EYE]
            crop_right, crop_left = eyesExtractor(rgb_img, right_coords, left_coords)
            # cv.imshow('right', crop_right)
            # cv.imshow('left', crop_left)
            eye_position, color = EyePositionEstimator(crop_right)
            colorBackgroundText(rgb_img, f'R: {eye_position}', FONTS, 1.0, (40, 220), 2, color[0], color[1], 8, 8)
            eye_position_left, color = EyePositionEstimator(crop_left)
            colorBackgroundText(rgb_img, f'L: {eye_position_left}', FONTS, 1.0, (40, 320), 2, color[0], color[1], 8, 8)
            
        end_time = time.time()-start_time
        fps = frame_counter/end_time

        frame =textWithBackground(rgb_img,f'FPS: {round(fps,1)}',FONTS, 1.0, (30, 50), bgOpacity=0.9, textThickness=2)
        
        final_img = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
        cv2.imshow('EYE_TRACKING', final_img)
        key = cv2.waitKey(2)
        if key==ord('q') or key ==ord('Q'):
            break
    cv2.destroyAllWindows()
    cam.release()   
                    
            
        














    
        


In [4]:
mp_draw = mp.solutions.drawing_utils
mp_holi = mp.solutions.holistic
# mp_face_mesh = mp.solutions.face_mesh
# face_mesh = mp_face_mesh.FaceMesh()
cap = cv2.VideoCapture(0)
with mp_holi.Holistic(min_detection_confidence=0.4,min_tracking_confidence=0.4) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()
        img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        res = holistic.process(img)
        # print(res.face_landmarks)
        img = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
        mp_draw.draw_landmarks(img,res.face_landmarks,mp_holi.FACEMESH_CONTOURS,
                               connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_contours_style())
        mp_draw.draw_landmarks(img,res.face_landmarks,mp_holi.FACEMESH_TESSELATION,
                               connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style())
        # mp_draw.draw_landmarks(img,res.face_landmarks,mp_holi.FACEMESH_IRISES,
        #                        connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_iris_style())
        cv2.imshow("Facial maps",img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# # cap3 = cv2.VideoCapture("C:/Users/Charanteja/OneDrive/Pictures/Desktop/Emotion_recognition/archive/emotion.mp4")
# cap3 = cv2.VideoCapture(0)

# while True:
#     ret,image = cap3.read()
#     if ret is not True:
#         break
#     h,w,_ = image.shape
#     # print("(Height,width) = ",h,w)
#     rgb_img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
#     res = face_mesh.process(rgb_img)
#     for facial_landmarks in res.multi_face_landmarks:
#         for i in range(0,468):
#             pt1 = facial_landmarks.landmark[i]
#             x = int(pt1.x * w)
#             y = int(pt1.y * h)
#             cv2.circle(image,(x,y),2,(100,100,0),-1)
#     cv2.imshow("Face_landmarks",image)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# cap3.release()
# cv2.destroyAllWindows()
mp_face_mesh = mp.solutions.face_mesh
# mp_draw = mp.solutions.drawing_utils
# mp_styles = mp.solutions.drawing_styles
# # cap2 = cv2.VideoCapture("C:/Users/Charanteja/OneDrive/Pictures/Desktop/Emotion_recognition/archive/emotion.mp4")
# cap2 = cv2.VideoCapture(0)

# while cap2.isOpened():
#     ret1,frame2 = cap2.read()
#     img2 = cv2.cvtColor(frame2,cv2.COLOR_RGB2BGR)
#     res2 = mp_face_mesh.FaceMesh(max_num_faces= 5,min_detection_confidence=0.5,min_tracking_confidence=0.5,refine_landmarks=True).process(img2)
#     img2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2RGB)
#     if res2.multi_face_landmarks:
#         for fl in res2.multi_face_landmarks:
#             mp_draw.draw_landmarks(
#                 image=img2,
#                 landmark_list = fl,
#                 connections = mp_face_mesh.FACEMESH_TESSELATION,
#                 landmark_drawing_spec = None,
#                 connection_drawing_spec = mp_styles.get_default_face_mesh_tesselation_style()
#             )
#             # mp_draw.draw_landmarks(
#             #     image=img2,
#             #     landmark_list = fl,
#             #     connections = mp_face_mesh.FACEMESH_CONTOURS,
#             #     landmark_drawing_spec = None,
#             #     connection_drawing_spec = mp_styles.get_default_face_mesh_contours_style()
#             # )
#             # mp_draw.draw_landmarks(
#             #     image=img2,
#             #     landmark_list = fl,
#             #     connections = mp_face_mesh.FACEMESH_IRISES,
#             #     landmark_drawing_spec = None,
#             #     connection_drawing_spec = mp_styles.get_default_face_mesh_iris_style()
#             # )
#     cv2.imshow("Face mesh",img2)
#     if cv2.waitKey(10) & 0xFF == ord("q"):
#         break
# cap2.release()
# cv2.destroyAllWindows()
    
    

    
            
    